#Project Title: Retrieval-Augmented QA with LangChain, Chroma & Wikipedia

Problem Statement:   

Traditional search systems often fail to deliver context-rich answers from unstructured documents. There's a growing need for modular, explainable RAG pipelines that integrate local knowledge with external sources like Wikipedia for more accurate and relevant responses.

Project Overview:

This notebook demonstrates a lightweight RAG-based QA system built using LangChain, Chroma, and Hugging Face models. It enables semantic search over uploaded documents and Wikipedia, optimizing retrieval latency and answer relevance through efficient chunking, embedding, and vector search.


Key Features:  

Semantic search using all-MiniLM-L6-v2 embeddings

Wikipedia integration via Python API

Vector store with Chroma and persistent storage

Modular LangChain pipeline for easy extension

Deployed in Colab with GPU support

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving extracted_text.txt to extracted_text.txt


In [ ]:
!pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=be218b48f9efe7106cb40d492d3e697cdac7f8f1f500948b32c3563f4af4b4be
  Stored in directory: /root/.cache/pip/wheels/8f/ab/cb/45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
Successfully built wikipedia


In [ ]:
!pip install langchain langchain-community langchain_openai langchain_chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.6/19.6 MB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 443.5/443.5 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 41.8 MB/s eta 0:00

#Base Model

In [ ]:
# 📦 Imports
import os
import gradio as gr
from transformers import pipeline
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import TextLoader, WikipediaLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 📄 Load documents: local + Wikipedia
local_loader = TextLoader("extracted_text.txt")  # 🔁 Replace with your actual file
wiki_loader = WikipediaLoader(query="New Delhi is the Capital of India", load_max_docs=2)

docs = local_loader.load() + wiki_loader.load()

# ✂️ Split documents into chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = splitter.split_documents(docs)

# 🔍 Embed and store in Chroma
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = Chroma.from_documents(split_docs, embedding_model)

# 🤖 Load local model with transformers
local_pipeline = pipeline("text2text-generation", model="google/flan-t5-base", max_length=256, truncation=True)
llm = HuggingFacePipeline(pipeline=local_pipeline)

# 🔗 Create RAG chain
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})
rag_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, return_source_documents=True)

# 🧠 Define response function
def generate_response(question):
    try:
        response = rag_chain.invoke({"query": question})
        answer = response["result"]
        sources = response.get("source_documents", [])

        if not sources:
            return "**Answer (fallback):**\nThis question is outside the scope of the provided documents."

        source_text = "\n\n".join([doc.page_content for doc in sources])
        return f"**Answer:**\n{answer}\n\n**Sources:**\n{source_text}"
    except Exception as e:
        return f"❌ Error: {str(e)}"


/tmp/ipython-input-2713280434.py:23: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0
/tmp/ipython-input-2713280434.py:28: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=local_pipeline)


Gradio Interface based on local document/file

In [ ]:
# 🎛️ Launch Gradio interface
iface = gr.Interface(
    fn=generate_response,
    inputs=gr.Textbox(lines=2, placeholder="Enter your question here..."),
    outputs="markdown",
    title="🌐 RAG QA with Local + Wikipedia Knowledge",
    description="Ask questions based on local documents and Wikipedia. Powered by LangChain, Chroma, and Hugging Face's flan-t5-base running locally."
)

iface.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://f0ceab5b9450506adc.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://f0ceab5b9450506adc.gradio.live
